## Welfare Range Calculations

### Set-Up

In [25]:
import squigglepy as sq
from squigglepy.numbers import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

N = 10*K
PERCENTILES = [1, 5, 25, 50, 75, 95, 99]

### P(Sentience) Distributions

#### Distribution Sampling

In [26]:
pigs = np.ones(N) - sq.sample(sq.lognorm(0.001, 0.2, rclip=.4), N)
chickens = np.ones(N) - sq.sample(sq.lognorm(0.05, 0.25, lclip=0.01, rclip=0.5), N)
carp = sq.sample(sq.norm(0.55, 0.85, lclip=0, rclip=1), N)
octopuses = np.ones(N) - sq.sample(sq.lognorm(0.1, 0.5, lclip=0.01, rclip=.99), N)
shrimp = sq.sample(sq.lognorm(0.2, 0.7, lclip = 0.01, rclip=1), N)
bsf = sq.sample(sq.norm(0.1, 0.6, lclip = 0.01, rclip=1), N)

models_dict = {"Pigs": pigs, "Chickens": chickens, "Carp": carp, "Octopuses": octopuses, "Shrimp": shrimp, "BSF": bsf}

#### P(Sentience) Summary Statistics

In [27]:
def get_percentiles_table(models_dict, animals):
    cols = ['1%', '5%', '25%', '50%', '75%', '95%', '99%', "Mean"]
    percentiles_dict = {}
    for animal in animals:
        percentiles = np.percentile(models_dict[animal], PERCENTILES)
        percent_and_mean = np.append(percentiles, np.mean(models_dict[animal]))
        percentiles_dict[animal] = percent_and_mean
    df = pd.DataFrame.from_dict(percentiles_dict, columns=cols, orient='index')
    print("Probability of Sentience")
    print(df)
    df.to_csv(os.path.join("outputs", "percentiles.csv"))
    return df

get_percentiles_table(models_dict, ["Pigs", "Chickens", "Carp", "Octopuses", "Shrimp", "BSF"])

Probability of Sentience
                 1%        5%       25%       50%       75%       95%  \
Pigs       0.600000  0.801040  0.958274  0.985785  0.995056  0.998989   
Chickens   0.652864  0.749757  0.845560  0.889252  0.919968  0.950575   
Carp       0.490148  0.549095  0.637261  0.698825  0.760671  0.848842   
Octopuses  0.296134  0.493661  0.685611  0.773634  0.838500  0.899745   
Shrimp     0.151631  0.198836  0.288645  0.373068  0.484491  0.697006   
BSF        0.010000  0.101413  0.248673  0.350277  0.454754  0.601108   

                99%      Mean  
Pigs       0.999665  0.957434  
Chickens   0.963617  0.874811  
Carp       0.906721  0.698600  
Octopuses  0.927535  0.745593  
Shrimp     0.918929  0.400870  
BSF        0.694999  0.351725  


,1%,5%,25%,50%,75%,95%,99%,Mean
Pigs,0.600000,0.801040,0.958274,0.985785,0.995056,0.998989,0.999665,0.957434
Chickens,0.652864,0.749757,0.845560,0.889252,0.919968,0.950575,0.963617,0.874811
Carp,0.490148,0.549095,0.637261,0.698825,0.760671,0.848842,0.906721,0.698600
Octopuses,0.296134,0.493661,0.685611,0.773634,0.838500,0.899745,0.927535,0.745593
Shrimp,0.151631,0.198836,0.288645,0.373068,0.484491,0.697006,0.918929,0.400870
BSF,0.010000,0.101413,0.248673,0.350277,0.454754,0.601108,0.694999,0.351725
